In [1]:
import jsonlines
import re
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import linear_model
from tqdm import tqdm

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"@", "", string)
    return string.lower()

In [3]:
count = 0
train_data = []
valid_data = []
test_data = []
with jsonlines.open('instances.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        count += 1
        if (count > 15630 and count <= 17584):
            valid_data.append(obj)
        if (count > 17584):
            test_data.append(obj)
        if(count<=15630):
            train_data.append(obj)

count = 0
truth_data = []
with jsonlines.open('truth.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        truth_data.append(obj)



In [6]:
train_data_df = pd.DataFrame.from_dict(train_data)
truth_data_df = pd.DataFrame.from_dict(truth_data)
train = pd.merge(train_data_df, truth_data_df, on="id")

In [7]:
data = train.values
print("Training data length", len(data))

Training data length 15630


In [9]:
df = []
y = []

In [10]:
for i in data:
    if(i[9]=="clickbait"):
        y.append(1)
    else:
        y.append(0)

textFeatures = ["postText", "targetCaptions", "targetParagraphs", "targetTitle", "targetKeywords", "targetDescription"]
textColumns = train[textFeatures]
textColumns = textColumns.values.tolist()

In [24]:
print(textColumns[0])

[['UK’s response to modern slavery leaving victims destitute while abusers go free'], ['modern-slavery-rex.jpg'], ['Thousands of modern slavery victims have\xa0not come forward, while others who have chosen to report their enslavers have ended up destitute as a result of insufficient support, say\xa0MPs', '“Inexcusable” failures in the UK’s system for dealing with modern slavery are\xa0leaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them, an alarming report has warned.', 'Thousands of\xa0victims\xa0have not come forward, while others who have chosen to give evidence against their enslavers have ended up destitute as a result of insufficient support, according to\xa0a report published\xa0today by\xa0the Work and Pensions Committee.', 'It is estimated there are between 10,000 and 13,000 victims of modern slavery in the UK, but the report\xa0warns that failings in the current mechanism mean\xa0that once they a

In [11]:
for i in range(len(textColumns)):
    text = []
    for j in range(0,5):
        k = textColumns[i][j]
        if (j == 3 or j == 4):
            text.append(k)
        else:
            text+=k
    words = ""
    for string in text:
        string = clean_str(string)
        words +=" ".join(string.split())
    df+=[words]

In [23]:
print(df[0])

uk s response to modern slavery leaving victims destitute while abusers go freemodern slavery rex jpgthousands of modern slavery victims have not come forward while others who have chosen to report their enslavers have ended up destitute as a result of insufficient support say mpsinexcusable failures in the uk s system for dealing with modern slavery are leaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them an alarming report has warnedthousands of victims have not come forward while others who have chosen to give evidence against their enslavers have ended up destitute as a result of insufficient support according to a report published today by the work and pensions committeeit is estimated there are between 10 000 and 13 000 victims of modern slavery in the uk but the report warns that failings in the current mechanism mean that once they are identified they have no automatic formal immigration status or r

In [32]:
vectorizer = CountVectorizer(input='content', lowercase=False, analyzer='word', stop_words='english')
X = vectorizer.fit_transform(df)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X)
print(X_train_tfidf.shape)

model = linear_model.LinearRegression()
model.fit(X_train_tfidf, y)

(15630, 375302)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [35]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
cross_val_score(model,X_train_tfidf , y, cv=6)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([-6.67997796, -8.83668459, -6.15611098, -7.53369423, -6.20383073,
       -1.29161184])

In [34]:
print((X_train_tfidf[0].shape))

(1, 375302)


In [13]:
print("Validation")
valid_data_df = pd.DataFrame.from_dict(valid_data)
valid = pd.merge(valid_data_df, truth_data_df, on="id")
vdata = valid.append(train).values

y_valid = []
for i in vdata:
    if (i[9] == "clickbait"):
        y_valid.append(1)
    if (i[9] == "no-clickbait"):
        y_valid.append(0)

y_valid = pd.DataFrame(y_valid)
print("Y_valid length", len(y_valid))
vdata = valid[textFeatures].append(train[textFeatures]).values.tolist()

Validation
Y_valid length 17584


In [14]:
df_valid = []
for i in range(len(vdata)):
    text = []
    for j in range(0, 5):
        k = vdata[i][j]
        if (j == 3 or j == 4):
            text.append(k)
        else:
            text += k
    words = ""
    for string in text:
        string = clean_str(string)
        words += " ".join(string.split())
    df_valid += [words]



In [37]:
# a_train, a_val, b_train, b_val = train_test_split(df_valid, y_valid, test_size = 0.11, random_state = 42)
predicted = []
for v in df_valid:
    valid_X = vectorizer.transform([v])
    X_valid_tfidf = tfidf_transformer.transform(valid_X)
    predicted.append(model.predict(X_valid_tfidf).round())

scores = accuracy_score(y_valid, predicted)
print("Validation Data Accuracy ", scores)

Validation Data Accuracy  0.9308462238398544


In [16]:
### TEST DATA ###

test_data_df = pd.DataFrame.from_dict(test_data)
test = pd.merge(test_data_df, truth_data_df, on="id")
tdata = test.values
print("length of test data")
print(len(tdata))

y_test =[]
df_test =[]

for i in tdata:
    if(i[9]=="clickbait"):
        y_test.append(1)
    if(i[9]=="no-clickbait"):
        y_test.append(0)

textColumns_test = test[textFeatures]
textColumns_test = textColumns_test.values.tolist()

for i in range(len(textColumns_test)):
    text = []
    for j in range(0,5):
        k = textColumns_test[i][j]
        if (j == 3 or j == 4):
            text.append(k)
        else:
            text+=k
    words = ""
    for string in text:
        string = clean_str(string)
        words +=" ".join(string.split())
    df_test+=[words]

length of test data
1954


In [17]:
predicted = []
for t in df_test:
    test_X = vectorizer.transform([t])
    X_test_tfidf = tfidf_transformer.transform(test_X)
    predicted.append(model.predict(X_test_tfidf).round())

scores = accuracy_score(y_test, predicted)
print("Test Data Accuracy ", scores)

Test Data Accuracy  0.4048106448311157


In [36]:
cross_val_predict(model,X_test_tfidf)

ValueError: Cannot have number of folds n_folds=3 greater than the number of samples: 1.